In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import json
import time
import random
import datetime
import re
import requests
from requests.auth import HTTPBasicAuth


In [12]:
API_TOKEN = "eccc953e-c1f7-45f4-a828-bc319440fd3a"

url = "https://api.sncf.com/v1/coverage/sncf/journeys"
params = {
    "from": "stop_area:SNCF:87686006",  # Paris Gare de Lyon
    "to": "stop_area:SNCF:87723197",    # Lyon Part-Dieu
    # "datetime": "20251001T080000"
}

response = requests.get(url, params=params, auth=HTTPBasicAuth(API_TOKEN, ""))
data = response.json()

for journey in data.get("journeys", []):
    departure = journey["departure_date_time"]
    arrival = journey["arrival_date_time"]
    duration = journey["duration"]
    print(f"Train departs {departure}, arrives {arrival}, duration {duration//60} minutes")


Train departs 20250930T205900, arrives 20250930T230800, duration 129 minutes


In [56]:
query = "Angers"
url = f"https://api.sncf.com/v1/coverage/sncf/places"
params = {"q": query}

response = requests.get(url, params=params, auth=HTTPBasicAuth(API_TOKEN, ""))
data = response.json()

In [13]:
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta

API_TOKEN = "eccc953e-c1f7-45f4-a828-bc319440fd3a"

def get_week_journeys(from_id, to_id, start_date):
    url = "https://api.sncf.com/v1/coverage/sncf/journeys"
    results = []
    dt = start_date

    for i in range(7):  # 7 days
        params = {
            "from": from_id,
            "to": to_id,
            "datetime": dt.strftime("%Y%m%dT%H%M%S")
        }
        r = requests.get(url, params=params, auth=HTTPBasicAuth(API_TOKEN, ""))
        data = r.json()
        journeys = data.get("journeys", [])
        for j in journeys:
            results.append(j)
        # move to next day at 00:00
        dt += timedelta(days=1)
    return results

from_station = "stop_area:SNCF:87437798"
to_station = "stop_area:SNCF:87202481"

start_date = datetime(2025, 10, 1, 8, 0, 0)  # start search date
week_trains = get_week_journeys(from_station, to_station, start_date)

# for t in week_trains:
#     print(t)

In [14]:
week_trains[0]

{'duration': 6989,
 'nb_transfers': 1,
 'departure_date_time': '20251005T155900',
 'arrival_date_time': '20251005T175529',
 'requested_date_time': '20251005T080000',
 'type': 'best',
 'status': '',
 'tags': ['walking', 'reliable', 'ecologic'],
 'co2_emission': {'value': 1600.0026, 'unit': 'gEC'},
 'air_pollutants': {'unit': 'g', 'values': {'nox': 0.0, 'pm': 0.0}},
 'durations': {'total': 6989,
  'walking': 209,
  'bike': 0,
  'car': 0,
  'ridesharing': 0,
  'taxi': 0},
 'distances': {'walking': 225,
  'bike': 0,
  'car': 0,
  'ridesharing': 0,
  'taxi': 0},
 'fare': {'found': False, 'total': {'value': '0.0'}, 'links': []},
 'calendars': [{'week_pattern': {'monday': False,
    'tuesday': False,
    'wednesday': False,
    'thursday': False,
    'friday': False,
    'saturday': False,
    'sunday': True},
   'active_periods': [{'begin': '20251005', 'end': '20251006'}]}],
 'sections': [{'id': 'QGJmSwG43fe5zAr52Kt8L8_0',
   'duration': 0,
   'co2_emission': {'value': 0.0, 'unit': 'gEC'},
 

In [ ]:
from math import radians, sin, cos, sqrt, asin
from typing import Any

def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on the Earth's surface."""
    R = 6371.0  # Earth's radius in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    return R * c
def compute_section_distance(sec:dict[str, Any]):
    """Compute distance of a section from geojson coordinates (km)."""
    coords = sec.get("geojson", {}).get("coordinates", [])
    if len(coords) < 2:
        return 0.0
    dist = 0.0
    for (lon1, lat1), (lon2, lat2) in zip(coords[:-1], coords[1:]):
        dist += haversine(lat1, lon1, lat2, lon2)
    return dist

def trip_stats(sections, print_steps=True):
    """Return totals + per-section breakdown:
    - total CO2 emissions (kgCO2)
    - total distance (km)
    - total time (s)
    - list of per-section stats with from/to
    If print_steps is True, prints a nice summary of each step.
    """
    total_co2 = 0.0
    total_dist = 0.0
    total_time = 0
    details = []

    if print_steps:
        print("==== Trajet Steps ====")

    for idx, sec in enumerate(sections):
        sec_time = sec.get("duration", 0)
        sec_dist = compute_section_distance(sec)
        sec_co2 = sec.get("co2_emission", {}).get("value", 0.0) / 1000

        # Update totals
        total_time += sec_time
        total_dist += sec_dist
        total_co2 += sec_co2

        # Names (fallback to empty string if missing)
        from_name = sec.get("from", {}).get("name", "")
        to_name = sec.get("to", {}).get("name", "")
        sec_type = sec.get("type", "unknown")

        # Save section detail
        details.append({
            "from": from_name,
            "to": to_name,
            "type": sec_type,
            "distance_km": sec_dist,
            "co2_kg": sec_co2,
            "time_s": sec_time
        })

        if print_steps:
            print(f"Step {idx+1}: {from_name} → {to_name}")
            print(f"  Type: {sec_type}")
            print(f"  Distance: {sec_dist:.2f} km")
            print(f"  Duration: {sec_time//60} min {sec_time%60} s")
            print(f"  CO2: {sec_co2:.3f} kg")
            print("-" * 30)

    if print_steps:
        print("==== Totals ====")
        print(f"Total distance: {total_dist:.2f} km")
        print(f"Total duration: {total_time//3600} h {(total_time%3600)//60} min")
        print(f"Total CO2: {total_co2:.3f} kg")
        print("================")

    return {
        "carbon_emission_kgCO2": total_co2,
        "distance_km": total_dist,
        "duration_s": total_time,
        "details": details
    }


In [ ]:
# Find the fastest train (minimum total_time)
fastest_train = min(week_trains, key=lambda train: trip_stats(train["sections"])["duration_s"])
trip_stats(fastest_train["sections"])

==== Trajet Steps ====
Step 1: La Rochelle Porte Dauphine (La Rochelle) → La Rochelle Porte Dauphine (La Rochelle)
  Type: crow_fly
  Distance: 0.00 km
  Duration: 0 min 0 s
  CO2: 0.000 kg
------------------------------
Step 2: La Rochelle Porte Dauphine (La Rochelle) → La Rochelle (La Rochelle)
  Type: public_transport
  Distance: 1.72 km
  Duration: 5 min 0 s
  CO2: 0.020 kg
------------------------------
Step 3:  → 
  Type: waiting
  Distance: 0.00 km
  Duration: 23 min 0 s
  CO2: 0.000 kg
------------------------------
Step 4: La Rochelle (La Rochelle) → Poitiers (Poitiers)
  Type: public_transport
  Distance: 132.69 km
  Duration: 85 min 0 s
  CO2: 1.580 kg
------------------------------
Step 5: Poitiers (Poitiers) → Poitiers - Gare Routière (Poitiers)
  Type: street_network
  Distance: 0.24 km
  Duration: 3 min 29 s
  CO2: 0.000 kg
------------------------------
==== Totals ====
Total distance: 134.65 km
Total duration: 1 h 56 min
Total CO2: 1.600 kg
==== Trajet Steps ====
Step 

{'carbon_emission_kgCO2': 1.6000026,
 'distance_km': 134.65347496727486,
 'duration_s': 6989,
 'details': [{'from': 'La Rochelle Porte Dauphine (La Rochelle)',
   'to': 'La Rochelle Porte Dauphine (La Rochelle)',
   'type': 'crow_fly',
   'distance_km': 0.0,
   'co2_kg': 0.0,
   'time_s': 0},
  {'from': 'La Rochelle Porte Dauphine (La Rochelle)',
   'to': 'La Rochelle (La Rochelle)',
   'type': 'public_transport',
   'distance_km': 1.7216768986493411,
   'co2_kg': 0.0204918,
   'time_s': 300},
  {'from': '',
   'to': '',
   'type': 'waiting',
   'distance_km': 0.0,
   'co2_kg': 0.0,
   'time_s': 1380},
  {'from': 'La Rochelle (La Rochelle)',
   'to': 'Poitiers (Poitiers)',
   'type': 'public_transport',
   'distance_km': 132.6949799907696,
   'co2_kg': 1.5795108,
   'time_s': 5100},
  {'from': 'Poitiers (Poitiers)',
   'to': 'Poitiers - Gare Routière (Poitiers)',
   'type': 'street_network',
   'distance_km': 0.2368180778559073,
   'co2_kg': 0.0,
   'time_s': 209}]}

In [10]:
import requests
import dotenv
import os
from math import radians, sin, cos, sqrt, asin
from requests.auth import HTTPBasicAuth


dotenv.load_dotenv()

sncf_api_key = os.getenv("SNCF_API_KEY")

def calculate_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """Calculate the great circle distance between two points in kilometers."""
    R = 6371.0  # Earth's radius in kilometers

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))  # Fixed the haversine formula

    return R * c

def _get_closest_station( city_query: str, lat: float, lon: float):
        """
        Returns the closest SNCF train station stop_area id given a latitude and longitude.
        """
        # if not, get closest station
        url = "https://api.sncf.com/v1/coverage/sncf/places"
        params = {"q": city_query}
        response = requests.get(url, params=params, auth=HTTPBasicAuth(sncf_api_key, ""), timeout=30)
        data = response.json()
        places = data.get("places", [])
        if not places:
            print(f"WARNING: No station found for {city_query}")
            return None, None
        closest_place = None
        min_distance = float('inf')
        for place in places:
            if "stop_area" in place["id"]:
                place_lat = float(place["stop_area"]["coord"]["lat"])
                place_lon = float(place["stop_area"]["coord"]["lon"])
                distance = calculate_distance(lat, lon, place_lat, place_lon)
                print(f"distance: {distance} from {city_query} to {place['stop_area']['name']}")
                if distance < min_distance:
                    min_distance = distance
                    closest_place = place
        if closest_place:
            stop_area_id = closest_place["stop_area"]["id"]
            stop_area_name = closest_place["stop_area"]["name"]
        else:
            print(f"WARNING: No station found for {city_query}")
            return None, None
        return stop_area_id, stop_area_name

In [12]:
_get_closest_station("Poitiers", 46.583333, 0.333333)

distance: 0.12334790391906102 from Poitiers to Poitiers
distance: 0.08029350373166526 from Poitiers to Poitiers - Gare Routière
distance: 535.2592580392784 from Poitiers to Yutz Poitiers


('stop_area:SNCF:87202481', 'Poitiers - Gare Routière')

In [ ]:
5280*

21120

In [ ]:
fastest_train["sections"][]

{'id': '27pMQxfk3ZswLrAd2kJ4aU_0',
 'duration': 0,
 'co2_emission': {'value': 0.0, 'unit': 'gEC'},
 'departure_date_time': '20251001T090500',
 'arrival_date_time': '20251001T090500',
 'to': {'id': 'stop_point:SNCF:87471003:LongDistanceTrain',
  'name': 'Rennes (Rennes)',
  'quality': 0,
  'stop_point': {'id': 'stop_point:SNCF:87471003:LongDistanceTrain',
   'name': 'Rennes',
   'label': 'Rennes (Rennes)',
   'coord': {'lon': '-1.672744', 'lat': '48.103517'},
   'links': [],
   'administrative_regions': [{'id': 'admin:fr:35238',
     'name': 'Rennes',
     'level': 8,
     'zip_code': '35000;35700',
     'label': 'Rennes (35000-35700)',
     'insee': '35238',
     'coord': {'lon': '-1.6800198', 'lat': '48.1113387'}}],
   'stop_area': {'id': 'stop_area:SNCF:87471003',
    'name': 'Rennes',
    'codes': [{'type': 'source', 'value': '87471003'},
     {'type': 'uic', 'value': '87471003'}],
    'timezone': 'Europe/Paris',
    'label': 'Rennes (Rennes)',
    'coord': {'lon': '-1.672744', 'lat

In [90]:
CO2_emission = 0
distance = 0
duration = 0

for section in week_trains[0]["sections"]:
    CO2_emission += float(section["co2_emission"]["value"])
    # distance += section["distance"]
    duration += section["duration"]

print(CO2_emission, distance, duration)

520.90392 0 8790
